## Trading Marginal Analysis

This file does marginal analysis by looking at the performance of each model, and then removing each signal one at a time and seeing the change in performance. So it runs TradingAlgorithms.ipynb over and over again and reads in the stats that are saved.

In [ ]:
# This file is used in MarginalAnalysis. It keeps track of how important each signal is to the performance of the final result.
# It looks at the performance of the model, and then it looks at the the same model trained on the same data minus that signal
# and looks at the change in performance.
def display_stats():
    # TODO: some charts to see this delta over time
    # TODO: make this flexible depending on the number of models

    stats_list = []
    for i in range(4):
        curr_stats = {}
        for j in range(-1, num_signals):
            file = 'Output/' + str(time_date) + '/' + str(i) + '/' + strategy_info['brief descriptor'] + '_' + str(j) + '/backtest_stats.csv'
            curr_stats[j] = round(float(pd.read_csv(file, index_col=0).loc['alpha (ann)', :].iat[0]), 4)
        print("Original annualized alpha on model #" + str(i) + ":", curr_stats[-1])
        for j in range(num_signals):
            if curr_stats[-1] > curr_stats[j]:
                incdec = "a decrease"
                diff = round(curr_stats[-1] - curr_stats[j], 4)
            elif curr_stats[-1] < curr_stats[j]:
                incdec = "an increase"
                diff = round(curr_stats[j] - curr_stats[-1], 4)
            else:
                incdec = "a change"
                diff = 0
            print("\tPerformance minus", list(signal_label_dict_og.keys())[j], "is", str(curr_stats[j]) + ",", incdec, "of", diff) # TODO: turn into printf to be cleaner
        stats_list.append(curr_stats)
        if i != 3:
            print()
    # stats_list

In [ ]:
# If this is a test, uses price_data_test.csv and Fundamental_test.dta to create merged_test.dta to make sure base calculations are good
test_added = ''
while True:
    is_test_input = input('Is this a test? (Enter True or False) ')
    if is_test_input == "True":
        is_test = True
        break
    if is_test_input == "False":
        is_test = False
        break
    else:
        print('Try again.')

full_added = ''
if not is_test:
    while True:
        is_full_input = input('Is this the full data? (Enter True or False) ') # User prompt
        if is_full_input == "True":
            is_full = True
            break
        if is_full_input == "False":
            is_full = False
            break
        else:
            print('Try again.')
else:
    is_full = False

# This is the base model
print('None removed')
%run signal_info.py
signal_label_dict_og, strategy_info = create_signal_label_dict()
%run TradingAlgorithms.ipynb
print('\td')
missing_signal = 0

# This removes one signal at a time from the signal_label_dict and runs TradingAlgorithms with that smaller list
for k in signal_label_dict_og.keys():
    if k == 'Gen_Label':
        continue
    print(k)
    signal_label_dict = signal_label_dict_og.copy()
    del signal_label_dict[k]

    current_signals = signal_label_dict.keys()
    num_signals = len(list(signal_label_dict.keys()))
    min_accounting_lag = 1
    # Info about the strategy, used for ex-post statistics and output not the actual backtest
    strategy_info = {
        'brief descriptor': '7stock_{0}signal_monthly'.format(num_signals), 
        'plot descriptor': 'B/H/S Strategy, Equal-Weighted',
        'universe': 'Public US equities with accounting data',
        'signals': current_signals, #, measured at most recent earnings announcement??
        'trading rule': 'Buy "A" stocks, sell "F" stocks', # What kind of weight? equal, rank, value?
        'holding period': 'One month',
        'periods per year': 12,
        'time lag': 'Minimum of {0} days from announcement of quarterly earnings'.format(min_accounting_lag),
        'output folder name': 'Output'
    }

    %run TradingAlgorithms.ipynb
    print('\td')
    missing_signal += 1
missing_signal = -1

In [ ]:
%%capture cap --no-stderr
%run Background_Scripts/display_MA_stats.ipynb
with open(f'Output/{time_date}/output.txt', 'w') as file:
    file.write(cap.stdout)

In [ ]:
print(cap)